In [1]:
!pip install gdown
!pip install vit-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 446.1 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 949.5 kB/s eta 0:00:0031m1.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 505.7 kB/s eta 0:00:00 eta -:--:--


In [5]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [2]:

!gdown --id '1_K-A78LMsW1c0uOmFNBoGZEHEvrmaZsV'

/home/adil/anaconda3/envs/fr/lib/python3.12/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_K-A78LMsW1c0uOmFNBoGZEHEvrmaZsV
To: /media/adil/2 TB/face/1Dataset_Final.rar
100%|██████████████████████████████████████| 37.5M/37.5M [00:08<00:00, 4.63MB/s]


In [ ]:
# prompt: unrar my  /content/1Dataset_Final.rar

!unrar x /content/1Dataset_Final.rar


In [3]:
import os
dataset_path='./Dataset_Final'
classes=os.listdir(dataset_path)
# classes.sort()
classes

['sad', 'angry', 'happy', 'neutral', 'disgust', 'surprise', 'fear']

In [17]:
import numpy as np
import pandas as pd
import time
def get_img_df_path(dataset_path, classes):
  df=pd.DataFrame(columns=["img_paths","classes"])

  i = 0
  for c in classes:
    image_path=os.path.join(dataset_path,c)
    paths = os.listdir(image_path)
    for path in paths:
      path_to_img=os.path.join(image_path,path)
      df.loc[len(df.index)]=[path_to_img,c]


  return df

In [8]:
df=get_img_df_path(dataset_path, classes)
print(df.classes.unique())
print(df['img_paths'][0])
print(df['img_paths'][1272])

['sad' 'angry' 'happy' 'neutral' 'disgust' 'surprise' 'fear']
./Dataset_Final/sad/2717.jpg
./Dataset_Final/sad/20672.json


In [9]:
from sklearn.model_selection import train_test_split # for split data into test,train
X_train, X_test, y_train, y_test = train_test_split(df["img_paths"],df["classes"], test_size=0.20, random_state=42,stratify=df["classes"])


In [11]:
import shutil
def move_files(new_p,paths,classes):
  if os.path.exists(new_p)==False:
    os.mkdir(new_p)

  for index,path in enumerate(paths):
    new_path=f"./{new_p}/{classes[index]}/"
    if os.path.exists(new_path)==False:
      os.mkdir(new_path)
    shutil.move(path,f"{new_path}" )

move_files("train",X_train.values,y_train.values)
move_files("test",X_test.values,y_test.values)

In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import pandas as pd
import torchvision.models as models

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = ImageFolder(root='train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = ImageFolder(root='test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = trainset.classes
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [13]:
import pandas as pd

df=pd.DataFrame(columns=["Name","Accuracy","Recall","Precision","F1","TrainingTime"])
df.head()

,Name,Accuracy,Recall,Precision,F1,TrainingTime


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:

import tqdm
def train_and_evaluate_model(model, name, df, trainloader, testloader, device, epochs, classes):
    """
    Trains and evaluates a model.

    Args:
        model (nn.Module): The PyTorch model to train.
        name (str): Name of the model or experiment.
        df (pd.DataFrame): DataFrame to store metrics.
        trainloader (DataLoader): DataLoader for training data.
        testloader (DataLoader): DataLoader for testing data.
        device (torch.device): Device to run the model on.
        epochs (int): Number of training epochs.
        classes (list): List of class names.

    Returns:
        pd.DataFrame: Updated DataFrame with new metrics.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.to(device)

    # Training
    print(f"{name} training started.")
    start_time = time.time()

    for epoch in range(epochs):
        model.train()

        epoch_loss = 0
        num_batches = len(trainloader)

        # Progress bar for batches within the epoch
        with tqdm.tqdm(total=num_batches, desc=f"Epoch {epoch+1}/{epochs}", leave=False) as pbar:
            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                pbar.update(1)

        avg_loss = epoch_loss / num_batches
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds.")

    # Evaluation
    print("Testing started.")
    model.eval()
    predicteds, all_predictions, actuals = [], [], []

    with torch.no_grad():
        for images, labels in tqdm.tqdm(testloader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            probabilities = F.softmax(outputs, dim=1)
            all_predictions.extend(probabilities.cpu().numpy())
            _, predicted = torch.max(outputs.data, 1)
            actuals.extend(labels.cpu().numpy())
            predicteds.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(actuals, predicteds)
    f1 = f1_score(actuals, predicteds, average='macro')
    recall = recall_score(actuals, predicteds, average='macro')
    precision = precision_score(actuals, predicteds, average='macro')

    df.loc[len(df.index)] = [name, accuracy, recall, precision, f1, end_time - start_time]

    # Plot Confusion Matrix
    plot_confusion_matrix(actuals, predicteds, classes)

    # Plot ROC curve
    plot_roc_curve(actuals, all_predictions, classes)

    return df


In [20]:
# here I am importing model
from vit_pytorch import SimpleViT

epochs=5
vit = SimpleViT(
    image_size = 224,
    patch_size = 32,
    num_classes = len(classes),
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)
# Define your class names (labels) function
df = train_and_evaluate_model(vit, "VIT", df, trainloader, testloader, device, epochs, classes)
df.head()

VIT training started.


Epoch 1/5 - Loss: 1.8058


Epoch 2/5 - Loss: 1.7606


Epoch 3/5 - Loss: 1.7241


Epoch 4/5 - Loss: 1.6848


Epoch 5/5 - Loss: 1.6421
Training completed in 371.52 seconds.
Testing started.


Testing:   0%|                                          | 0/958 [00:00<?, ?it/s]


NameError: name 'F' is not defined